In [12]:
import openpyxl as op
import numpy as np
import pandas as pd
import re
import os, sys
import support as sp
import time
import glob
import datetime

### Import and clean files

In [15]:
from openpyxl import load_workbook, Workbook
from openpyxl.worksheet.datavalidation import DataValidation

# -This script will last for about 6,5 minutes-

# Open a file
raw_path = 'data/raw/'
in_path = 'data/in/'
raw_dirs = os.listdir( raw_path )
in_dirs = os.listdir( in_path )

#remove all duplicates in the folder
for index, file in enumerate(raw_dirs):
    try:
        if raw_dirs[index][:10] == raw_dirs[index+1][:10] and raw_dirs[index][-7:] != raw_dirs[index+1][-7:]:
            #print(dirs[index], dirs[index+1])
            raw_dirs.remove(raw_dirs[index+1])
    except:
        print('End of list reached. Length of list is:', len(raw_dirs))

# start loading each file one by one. if it fails it will try to convert to .xlsx (openpyxl cannot handle .xls)
start = time.time()
for file in raw_dirs:
    try:
        wb2 = load_workbook(raw_path + file)
        ws = wb2.active
    except:
        if file[-4:] == 'xlsx':
            print(file, 'has thrown an error')
        else:
            #print(file, "failed processing -> converting .xls to .xlsx...")
            wb2 = load_workbook(sp.convert(raw_path + file))
            ws = wb2.active
            #print("Finished")
            
    # Clean headers by removing alphanumerical chars, capital letters, and replacing whitespaces with underscore
    for col in ws.iter_cols(min_row=1, max_col=35, max_row=1):
        for cell in col:
            if cell.value is not None:
                try:
                    temp = re.sub(r'[^a-zA-Z\s]','', cell.value)
                    temp = temp.lower().rstrip()
                except:
                    print(file, cell.value)
                    break
                
                temp = re.sub(r"\s+", '_', temp)
                ws[cell.coordinate] = temp
    
    book = Workbook()
    new = book.active
    
    # save the file with cleaned headers to a new folder
    for row in ws.iter_rows(min_col=1, max_col=35, min_row=1, max_row=2000):
        new.append((cell.value for cell in row))
    
    if file[-3:] == 'xls':          
        book.save(in_path + file[:-4] + '_converted.xlsx')
        # print(file[:-4] + '_converted.xlsx saved')
    else:
        book.save(in_path + file)
        # print(file, 'saved')
    
end = time.time()
print('Done!')
print(end - start)

End of list reached. Length of list is: 265


e:\users\maarten\appdata\local\programs\python\python36\lib\site-packages\openpyxl\reader\worksheet.py:322: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Blue Monday_RH.xlsx has thrown an error
Done!
408.12634325027466


### Standardizing headers

In [16]:
headers_d = {'extreme_cu' : 'extremecu', 'title' : 'story_title', 'time_change' : ['timechange', 'temporal_change'],
             'spatial_change' : ['spatiallocation', 'spatial_location'], 'med_close' : ['mediumclose', 'medclose'],
             'panel_nr' : 'panel', 'close_up' : 'closeup', 'character_change' : 'characters', 'divisional' : 'divsional_montage',
             'actual_page' : 'page_in_book', 'coded_page' : 'page', 'base_framing' : ['base', 'baseframing'],
             'causal_change' : 'causalchange', 'subjective_viewpoint' : 'subj'}

in_path = 'data/in/'
in_dirs = os.listdir( in_path )

start = time.time()
for file in in_dirs:
    wb2 = load_workbook(in_path + file)
    ws = wb2.active
        
    for col in ws.iter_cols(min_row=1, max_col=35, max_row=1):
        for cell in col:
            for key in headers_d:
                if cell.value == headers_d[key]:
                    ws[cell.coordinate] = key
                    #print('dict1', cell.value, key)
                
                for value in headers_d[key]:
                    if cell.value == value:
                        ws[cell.coordinate] = key
                     #   print('dict2', cell.value, key)
                        
    wb2.save(in_path + file)
    
end = time.time()
print('Done!')
print(end - start)

Done!
34.399967670440674


### Merging files

In [17]:
all_data = pd.DataFrame()
drop = ['writer', 'genre', 'drawer', 'originalyear', 'pub','format', 'publisher', 
        'chapter', 'st_panel', 'up', 'magazine_title', 'volume', 'macromono', 'macromacro', 'upright', 
        'macromicro', 'original_year', 'schema_total']#year, issuechapter

count = 0
for f in glob.glob("data/in/*.xlsx"):
    df = pd.read_excel(f)
    count += 1
    all_data = all_data.append(df, ignore_index=False)
print(count)
all_data = all_data.drop(drop, axis=1, errors='ignore')
all_data = all_data[all_data.title != 'SUM']
all_data = all_data[all_data.title != 'MEAN per panel']
all_data['id']=all_data.reset_index().index # place holder

e:\users\maarten\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


265


### Export merged file to .csv

In [18]:
all_data.to_csv('data/out/merged.csv')

# Data
### Cleaning data

In [43]:
data = pd.read_csv('data/out/merged.csv')
addit = pd.read_excel('Raw/CCF_datameans.xlsx')


data['amb'] = data['amb'] + data['ambiguous']
addit = addit.rename({'Title': 'title'}, axis='columns')

data = data.fillna(0)
for i in range(len(data['time_change'])):
    if i == len(data['time_change'])-1:
        print('Completed')
        break
    else:    
        if data.loc[i,'title'] != data.loc[i+1,'title'] and data.loc[i+1, 'title'] == 0 or data.loc[i+1, 'title'] == '':
            data.loc[i+1, 'title'] = data.loc[i, 'title']
        if data.loc[i,'issuechapter'] != data.loc[i+1,'issuechapter'] and data.loc[i+1, 'issuechapter'] == 0 or data.loc[i+1, 'issuechapter'] == '':
            data.loc[i+1, 'issuechapter'] = data.loc[i, 'issuechapter']

        a = str(data.loc[i, 'title'])
        b = str(data.loc[i+1, 'title'])
        c = str(data.loc[i, 'issuechapter'])
        d = str(data.loc[i+1, 'issuechapter'])
        if a[-5:] == b[-5:]:
            data.loc[i+1, 'title'] = data.loc[i, 'title']
        if a[:5] == b[:5]:
            data.loc[i+1, 'title'] = data.loc[i, 'title'] 
        if c[-2:] == d[-2:] or c[:5] == d[:5]:
            data.loc[i+1, 'issuechapter'] = data.loc[i, 'issuechapter']
#        if c[:5] == d[:5]:
 #           data.loc[i+1, 'issuechapter'] = data.loc[i, 'issuechapter']
        if c.isdigit() and d.isdigit() and len(c) < 3:
            if c != d:
                data.loc[i+1, 'issuechapter'] = data.loc[i, 'issuechapter']
            
        if len(str(data.loc[i, 'coded_page'])) >= 4 or data.loc[i, 'coded_page'] == 0:
            data.loc[i, 'coded_page'] = data.loc[i-1, 'coded_page']
        
        #if isintance(data.loc[i, 'macro'], datetime.datetime)
#data['title_id'] = data['title'] + data['issuechapter']
data["combo"] = data["title"] + data["issuechapter"].map(str)
data['id'] = (data['combo']).astype('category').cat.codes

e:\users\maarten\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,1,20,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Completed


In [42]:
uniq, countser = np.unique(addit['Country_notype'], return_counts=True)
print(uniq, sum(countser))
countser

['American' 'Chinese' 'Dutch' 'Flemish' 'French' 'German' 'Japanese'
 'Korean' 'Spanish' 'Swedish'] 291


array([78, 11, 40, 40, 21, 10, 55, 15,  1, 20], dtype=int64)

In [45]:
#data['id'] = (data['title'] + '_' + str(data['issuechapter'])).astype('category').cat.codes
#print(len(data['time_change']))
print(np.unique(data['id']))
#print(len(data['issuechapter'].unique()))
len(data['id'].unique())


[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

274

### Merging dataframes and saving to .csv

In [46]:
mergeddf = pd.merge(data, addit, how="inner", on='title')
print(mergeddf.head())

  Unnamed: 0 actual_page  amb  amb.1  ambiguous  ambiguous.1  amorphic  \
0          2           0  0.0    0.0        1.0          0.0       0.0   
1          3           0  0.0    0.0        0.0          0.0       0.0   
2          4           0  0.0    0.0        0.0          0.0       0.0   
3          5           0  0.0    0.0        0.0          0.0       0.0   
4          6           0  0.0    0.0        0.0          0.0       0.0   

   base_framing causal_change character_change     ...       Panels/page  \
0           1.0             x                x     ...          4.733333   
1           1.0             0                1     ...          4.733333   
2           1.0             0                1     ...          4.733333   
3           1.0             1              0.5     ...          4.733333   
4           1.0             0                0     ...          4.733333   

      Macro      Mono     Micro  Amorphic       Amb  Characters  \
0  0.397817  0.493155  0.021032

In [49]:
mergeddf.to_csv('data/out/df.csv')

In [48]:
len(mergeddf['combo'].unique())
#mergeddf['Decade'].unique()

254